In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Load the saved model
model = tf.keras.models.load_model('model/model_working.h5')

In [2]:
#show the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 60, 60, 128)       73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 30, 30, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 115200)           

In [3]:
# Function to preprocess a single image
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (256, 256))
    image = image / 255.0
    return image

# Function to create rectangles around detected objects
def create_rectangles(image, mask):
    # Find contours in the binary mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw rectangles around the detected objects
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green rectangle with 2-pixel width
    
    return image

# Function to check if the object is detected
def is_object_detected(mask, threshold=0.5):
    return np.max(mask) > threshold

# Directory containing the new unseen images
new_images_dir = "img/test/"

# Loop through the new images and make predictions
for image_file in os.listdir(new_images_dir):
    image_path = os.path.join(new_images_dir, image_file)
    
    # Preprocess the image
    image = preprocess_image(image_path)
    
    # Make a prediction using the model
    mask = model.predict(np.expand_dims(image, axis=0))[0]
    
    # Threshold the mask (if needed)
    threshold = 0.5
    binary_mask = (mask > threshold).astype(np.uint8)
    
    # Create rectangles around the objects in the original image
    image_with_rectangles = create_rectangles(image.copy(), binary_mask)
    
    # Check if the desirable object is detected
    if is_object_detected(binary_mask):
        print(f"Desirable object is detected in {image_file}.")
    else:
        print(f"Desirable object is NOT detected in {image_file}.")
    
    # Display the image with rectangles
    cv2.imshow("Image with Rectangles", image_with_rectangles)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Desirable object is detected in Screenshot_1.png.
Desirable object is detected in Screenshot_10.png.
Desirable object is NOT detected in Screenshot_11.png.
Desirable object is detected in Screenshot_2.png.
Desirable object is detected in Screenshot_3.png.
Desirable object is detected in Screenshot_4.png.
Desirable object is detected in Screenshot_5.png.
Desirable object is detected in Screenshot_6.png.
Desirable object is detected in Screenshot_7.png.
Desirable object is detected in Screenshot_8.png.
Desirable object is detected in Screenshot_9.png.
